# Decision Trees

Code is free of any AI-generation, is completely organic, GMO-Free, Pesticide free and has no added preservatives

In [124]:
import numpy as np

In [ ]:
# Given Dataset
data = [
    [12.0, 1.5, 1, 'Wine'],
    [5.0, 2.0, 0, 'Beer'],
    [40.0, 0.0, 1, 'Whiskey'],
    [13.5, 1.2, 1, 'Wine'],
    [4.5, 1.8, 0, 'Beer'],
    [38.0, 0.1, 1, 'Whiskey'],
    [11.5, 1.7, 1, 'Wine'],
    [5.5, 2.3, 0, 'Beer']
]

test_data = np.array([
    [6.0, 2.1, 0],   # Expected: Beer
    [39.0, 0.05, 1], # Expected: Whiskey
    [13.0, 1.3, 1]   # Expected: Wine
])

In [126]:
# Takes an integer and returns a label
def encode_class_index(class_index):
    match class_index:
        case 'Beer': return 0
        case 'Whiskey': return 1
        case 'Wine': return 2
        case _: raise Exception("Unknown Class")

# Takes an integer and returns a label
def decode_class_index(class_index):
    match class_index:
        case 0: return 'Beer'
        case 1: return 'Whiskey'
        case 2: return 'Wine'
        case _: raise Exception("Unknown Class")

In [ ]:
# Splitting data into features and labels
X = np.array(tuple(data_point[:-1] for data_point in data))
Y = np.array(tuple(encode_class_index(data_point[-1]) for data_point in data))

In [128]:
# Calculate the gini impurity, given an array of labels
def gini(labels):
    _, distribution = np.unique(labels, return_counts=True)
    distribution = distribution/distribution.sum()

    return 1 - (distribution**2).sum()

# Calculate the gini impurity, given an array of labels
def entropy(labels):
    _, distribution = np.unique(labels, return_counts=True)
    distribution = distribution/distribution.sum()

    return -(distribution * np.log2(distribution)).sum()

In [ ]:
# The Node class for the Decision Tree
class Node:

    # max_depth and metric need to be defined when instantiating root node
    # It will be passed down to appropriate children
    # metric can be either "gini" or "entropy"
    def __init__(self, max_depth = 5, metric = 'gini'):
        self.metric = metric
        self.max_depth = max_depth

        # Decision Nodes will have a condition as well as a left and right child reference
        self.feature_index = None 
        self.threshold = None
        self.left = None  
        self.right = None 

        # Leaf nodes will have a value stored
        self.value = None 

    def _predict(self, X):
        # If This is a leaf node, return stored value
        if self.value is not None: return self.value

        # Otherwise, apply the condition and go down to appropriate child 
        if (X[self.feature_index] < self.threshold): return self.left._predict(X)
        else: return self.right._predict(X)  
    
    # Utility Wrapper function so you can pass in a single test point or multiple points
    def predict(self, X):
        if len(X.shape) > 0:
            return np.array([
                self._predict(point) for point in X
            ])
        else:
            return self._predict(X)
    
    # Learning Algorithm using gini impurity
    def _split_gini(self, X_train, Y_train):
        impurity = gini(Y_train)

        # Should this be a leaf node?
        if impurity == 0 or self.max_depth == 1:
            self.value = Y_train[0]
            return
        
        # Find the best (feature,threshold) pair to split on
        least_impurity = 2
        best_feature = -1
        best_threshold = -1


        for feature in range(len(X_train[0])):
            for threshold in X_train[:, feature]:
                mask = X_train[:, feature] >= threshold
                impurity_R = gini(Y_train[mask]) * mask.sum()
                impurity_L = gini(Y_train[~ mask]) * (~ mask).sum()
                impurity = (impurity_L + impurity_R)/len(mask)
                if impurity < least_impurity:
                    least_impurity = impurity
                    best_feature = feature
                    best_threshold = threshold

        # Store condition
        self.feature_index = best_feature
        self.threshold = best_threshold

        # Create child nodes
        self.left = Node(max_depth=self.max_depth-1, metric=self.metric)
        self.right = Node(max_depth=self.max_depth-1, metric=self.metric)

        # Recursively make the child nodes learn
        mask = X_train[:, self.feature_index] >= self.threshold
        self.right.split(X_train[mask], Y_train[mask])
        self.left.split(X_train[~ mask], Y_train[~ mask])

    def _split_entropy(self, X_train, Y_train):
        raise Exception("Not implemented")

    # Have the decision tree learn using the training data passed
    # Utility wrapper function that calls the right implementation
    def split(self, X_train, Y_train):
        if self.metric == 'gini': self._split_gini(X_train, Y_train)
        elif self.metric == 'entropy': self._split_gini(X_train, Y_train)
        else: raise Exception("Unknown Metric "+ str(self.metric))

    # Pretty print the decision tree
    # Works by recursively printing the tree in a DFS style 
    # while passing down indentation information
    def pretty_print(self, feature_names, depth=0):        
        if self.value is not None:
            print("|   "*(depth-1) + "    ", end="")
            print("Prediction is:", self.value)
        
        else:
            print("|   "*depth, end="")
            print(f"IF {feature_names[self.feature_index]} >= {self.threshold}:")
            self.right.pretty_print(feature_names, depth+1)
            print("|   "*depth, end="")
            print(f"ELSE:")
            self.left.pretty_print(feature_names, depth+1)

        if (depth == 0): print()

In [ ]:
# Create a Decision tree and fit it to the train dataset
model = Node()
model.split(X,Y)

# Pretty print the decision tree
model.pretty_print(["Alcohol", 'Sugar', "Color"])

# Evaluate on the test set
print(tuple(decode_class_index(x) for x in model.predict(test_data)))

IF Alcohol >= 11.5:
|   IF Alcohol >= 38.0:
|       Prediction is: 1
|   ELSE:
|       Prediction is: 2
ELSE:
    Prediction is: 0

('Beer', 'Whiskey', 'Wine')
